## 🥉 Camada Bronze – Coleta e Validações


#### 📌 Passo 1: Criação do Schema
###### Estrutura medalhão então criei 3 Schemas (Bronze/Silver/Gold)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS daniel_bronze;
CREATE SCHEMA IF NOT EXISTS daniel_silver;
CREATE SCHEMA IF NOT EXISTS daniel_gold;


#### 📌 Passo 2: Carregue as variáveis
###### Carregar variáveis de acesso sem expor no código

In [0]:
# Carregar variáveis de acesso sem expor no código
%run "./Camada - Configuração"

#### 📌 Passo 3: Ingestão do Mongodb

###### Alternativa do momento para conseguir extrair o Mongodb foi usar Python com o Spark 

In [0]:
# Importações
import pymongo
import pandas as pd

# Conectar no MongoDB usando as variáveis carregadas
client = pymongo.MongoClient(
    f"mongodb://{mongodb_user}:{mongodb_pwd}@{mongodb_ip}:27017/?authSource=admin"
)
db = client[mongodb_db]
collection = db[mongodb_collection]

# Buscar os dados
data = list(collection.find())
df_pd = pd.DataFrame(data)

# Corrigir _id
if "_id" in df_pd.columns:
    df_pd["_id"] = df_pd["_id"].astype(str)

# Converter para Spark
df_trasacoes = spark.createDataFrame(df_pd)

# (Opcional) Remover a Tabela antiga visando Evitar Conflitos
spark.sql("DROP TABLE IF EXISTS adb_cliente_savana_prd.daniel_bronze.transacoes")

# Salvar na Bronze
df_trasacoes.write.format("delta").mode("overwrite") \
    .saveAsTable("adb_cliente_savana_prd.daniel_bronze.transacoes")

# Visualizar
display(df_trasacoes)



#### 📌 Passo 4: Ingestão do SQL

###### Com problemas de erros estou fazendo o mesmo processo do Mongodb, ultilizando o Python com o Spark

In [0]:
# Importações
import pymysql
import pandas as pd
from pyspark.sql.functions import to_date, col

# Conectar ao MySQL usando as variáveis
conn = pymysql.connect(
    host=sql_ip,
    user=sql_user,
    password=sql_pwd,
    database=sql_database,
    port=3306
)

# Leitura com Pandas
df_pd = pd.read_sql(f"SELECT * FROM {sql_table}", conn)

# Conversão para Spark
df_clientes = spark.createDataFrame(df_pd)

# (Opcional) Remover a Tabela antiga visando Evitar Conflitos
spark.sql("DROP TABLE IF EXISTS adb_cliente_savana_prd.daniel_bronze.clientes")

# Salvar na Bronze
df_clientes.write.format("delta").mode("overwrite") \
    .saveAsTable("adb_cliente_savana_prd.daniel_bronze.clientes")

# Visualizar
display(df_clientes)